In [4]:
import cv2
import numpy as np

# Load image
image_path = r"C:\Users\ramkumar\OneDrive - BI Worldwide\Desktop\hackthon\MRP.jpg"  # Replace with your image path
image = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect circles or squares using HoughCircles or contour detection
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=100, param1=50, param2=30, minRadius=10, maxRadius=20)

# Assuming circles are found, extract their coordinates
if circles is not None:
    circles = np.round(circles[0, :]).astype("int")
    for (x, y, r) in circles:
        # Draw the circle in the output image
        cv2.circle(image, (x, y), r, (0, 255, 0), 4)

# Now we have the position of the reference circles/squares



In [12]:
# Get pixel distance between two reference circles
pixel_distance = np.linalg.norm(np.array([ circles]))

# Calculate pixel to cm conversion factor
cm_per_pixel = 10 / pixel_distance  # Since we know the real-world distance is 10 cm

print(cm_per_pixel)

# Now detect the object and calculate its size in cm


0.006533518573697174


In [2]:
#camera access for laptop 

import cv2

def turn_on_camera():
    # Turn on the system camera (open the video capture)
    cap = cv2.VideoCapture(0)  # '0' is the default webcam, change it if you have multiple cameras

    if not cap.isOpened():
        print("Error: Camera could not be opened.")
        return None
    
    print("Camera is now ON")

    while True:
        ret, frame = cap.read()

        if not ret:
            print("Failed to grab frame")
            break

        # Display the camera feed
        cv2.imshow('Camera Feed', frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Turning off the camera...")
            break

    # Turn off the camera (release the capture)
    cap.release()
    cv2.destroyAllWindows()

def turn_off_camera():
    # You can simulate turning off by releasing the camera capture object
    print("Camera is now OFF")

# Usage
turn_on_camera()  # This will turn on the camera and display the feed

# You can call `turn_off_camera()` after closing the feed with 'q', but it's not necessary


Camera is now ON
Turning off the camera...
